In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

RANDOM_STATE = 42

torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed_all(RANDOM_STATE)  # For multi-GPU.
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)


def _init_fn(worker_id):
    np.random.seed(int(RANDOM_STATE))

In [3]:
train = pd.read_csv('../data/train.csv', header=None, names=["text"])
test = pd.read_csv('../data/answers.csv', header=None, names=["text"])


def separate_xy(df, prefix="train"):
    df[prefix + "_X"] = df["text"].apply(lambda x: x[:-1])
    df[prefix + "_y"] = df["text"].apply(lambda x: x[-1])
    return df

train_df = separate_xy(train, 'train')
test_df = separate_xy(test, 'test')

In [4]:
train_df

,text,train_X,train_y
0,SR2SR2SGR,SR2SR2SG,R
1,SRGSRSRSR,SRGSRSRS,R
2,KFKFKFKFK,KFKFKFKF,K
3,X2LS2FRS2,X2LS2FRS,2
4,MMMMGGGGG,MMMMGGGG,G
...,...,...,...
2538,S2ZR22222,S2ZR2222,2
2539,62SNRM1MM,62SNRM1M,M
2540,ASSRSRSRS,ASSRSRSR,S
2541,2ZZT2Z2ZT,2ZZT2Z2Z,T


In [5]:
train_X, val_X, train_y, val_y = train_test_split(train_df["train_X"], train_df["train_y"], test_size=0.1, random_state=RANDOM_STATE)

In [6]:
vocab = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567'

id2char = {idx: char for idx, char in enumerate(vocab)}
char2id = {char: idx for idx, char in enumerate(vocab)}


# Define the size of the input (number of unique characters), hidden layer, and output (same as input)
input_size = len(vocab)
hidden_size = len(vocab) * 3
output_size = len(vocab)

In [7]:

# Define the RNN model
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_hidden_layers=1, embedding_dim=10):
        super(SimpleLSTM, self).__init__()
        self.hidden_size = hidden_size

        # Embedding layer: 8 characters in the alphabet and embedding_dim can be a small number like 10
        self.embedding = nn.Embedding(input_size, embedding_dim=embedding_dim)
        
        # RNN layer
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_hidden_layers, batch_first=True)

        # Output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Embedding input
        embedded = self.embedding(x)

        # RNN output
        lstm_out, _ = self.lstm(embedded)
        last_lstm_out = lstm_out[:, -1, :]

        # Fully connected layer output
        out = self.fc(last_lstm_out)
        return out

In [8]:

# Create the model
simple_rnn = SimpleLSTM(input_size, hidden_size, output_size)

# Define the loss function and the optimizer
# loss_function = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
model_output = simple_rnn(torch.tensor([[18, 18, 18, 18]], dtype=torch.long))
model_output

tensor([[-0.0763, -0.0340, -0.0236, -0.0842, -0.0150, -0.0044, -0.0131, -0.0194,
          0.0457, -0.0388,  0.0771, -0.0409,  0.0475,  0.1285, -0.0146,  0.0720,
         -0.1960, -0.0074, -0.0617, -0.0261, -0.1313,  0.0337,  0.0061,  0.0036,
         -0.0881,  0.0635, -0.1149,  0.0259,  0.1789, -0.0761,  0.0471,  0.0518,
         -0.0643]], grad_fn=<AddmmBackward0>)

In [10]:
probabilities = F.softmax(model_output, dim=1)
probabilities

tensor([[0.0283, 0.0295, 0.0298, 0.0281, 0.0301, 0.0304, 0.0301, 0.0300, 0.0320,
         0.0294, 0.0330, 0.0293, 0.0320, 0.0347, 0.0301, 0.0328, 0.0251, 0.0303,
         0.0287, 0.0298, 0.0268, 0.0316, 0.0307, 0.0307, 0.0280, 0.0325, 0.0272,
         0.0313, 0.0365, 0.0283, 0.0320, 0.0322, 0.0286]],
       grad_fn=<SoftmaxBackward0>)

In [11]:
id2char[int(probabilities.argmax())]

'3'

In [12]:
char2id['S']

18

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader

class CharSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        """
        Args:
            sequences (list of strings): List of character sequences.
            labels (list of strings): List of single characters (labels).
        """
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        # Convert the sequence and label to numeric format
        sequence = torch.tensor([char2id[char] for char in self.sequences[idx]], dtype=torch.long)
        label = torch.tensor(char2id[self.labels[idx]], dtype=torch.long)

        return sequence, label


In [14]:
train_X

2240    2SR22SR2
678     M2SKKZEE
1194    S2LZTK2K
1572    2MSG2M2M
2276    EIVD12E1
          ...   
1638    1111SRER
1095    111R11SR
1130    SSGSRSRS
1294    MZTKE2XZ
860     22MSR1SE
Name: train_X, Length: 2288, dtype: object

In [15]:
train_df["train_X"]

0       SR2SR2SG
1       SRGSRSRS
2       KFKFKFKF
3       X2LS2FRS
4       MMMMGGGG
          ...   
2538    S2ZR2222
2539    62SNRM1M
2540    ASSRSRSR
2541    2ZZT2Z2Z
2542    22MMMZTS
Name: train_X, Length: 2543, dtype: object

In [16]:
train_dataset = CharSequenceDataset(list(train_X), list(train_y))
# train_dataset = CharSequenceDataset(train_df["train_X"], train_df["train_y"])
val_dataset = CharSequenceDataset(list(val_X), list(val_y))
test_dataset = CharSequenceDataset(test_df["test_X"], test_df["test_y"])

train_dataset, val_dataset, test_dataset

(<__main__.CharSequenceDataset at 0x7f24f86b5450>,
 <__main__.CharSequenceDataset at 0x7f24f86c35d0>)

In [17]:
# Define batch size
batch_size = 64

# Create the DataLoader for training data
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=_init_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=_init_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=_init_fn)

train_loader, val_loader, test_loader

(<torch.utils.data.dataloader.DataLoader at 0x7f24f90fbe90>,
 <torch.utils.data.dataloader.DataLoader at 0x7f24f86cc750>)

In [18]:

# Best params: {'hidden_size': 128, 'learning_rate': 0.0002, 'embedding_dim': 50, 'num_hidden_layers': 1}
# Hyperparameters
hidden_size = 128
embedding_dim = 100
num_hidden_layers = 1
learning_rate = 0.0004
num_epochs = 50


model = SimpleLSTM(input_size, hidden_size, output_size, num_hidden_layers, embedding_dim)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [19]:

def train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer, model_save_path):
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for batch_idx, (data, targets) in enumerate(train_loader):
            # Forward pass
            scores = model(data)
            loss = criterion(scores, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data, targets in val_loader:
                scores = model(data)
                loss = criterion(scores, targets)
                val_loss += loss.item()

        val_loss /= len(val_loader)

        # Print training and validation loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

        # Check if this is the best model so far and save it if it is
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_save_path)
            print(f"Saved model with validation loss: {best_val_loss:.4f}")

# Example usage
model_save_path = '../trained_models/best_model.pth'  # Define your model save path
train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer, model_save_path)


Epoch [1/50], Training Loss: 3.2176, Validation Loss: 2.7712
Saved model with validation loss: 2.7712
Epoch [2/50], Training Loss: 2.5353, Validation Loss: 2.3419
Saved model with validation loss: 2.3419
Epoch [3/50], Training Loss: 2.2987, Validation Loss: 2.1816
Saved model with validation loss: 2.1816
Epoch [4/50], Training Loss: 2.1698, Validation Loss: 2.0861
Saved model with validation loss: 2.0861
Epoch [5/50], Training Loss: 2.0753, Validation Loss: 2.0221
Saved model with validation loss: 2.0221
Epoch [6/50], Training Loss: 2.0023, Validation Loss: 1.9670
Saved model with validation loss: 1.9670
Epoch [7/50], Training Loss: 1.9480, Validation Loss: 1.9244
Saved model with validation loss: 1.9244
Epoch [8/50], Training Loss: 1.9005, Validation Loss: 1.8930
Saved model with validation loss: 1.8930
Epoch [9/50], Training Loss: 1.8491, Validation Loss: 1.8527
Saved model with validation loss: 1.8527
Epoch [10/50], Training Loss: 1.8106, Validation Loss: 1.8414
Saved model with val

In [20]:

def predict(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    actuals = []

    with torch.no_grad():
        for data, targets in data_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)

            predictions.extend(predicted.cpu().numpy())
            actuals.extend(targets.cpu().numpy())

    return predictions, actuals


In [21]:
predictions, actuals = predict(model, val_loader)

# Print the classification report
print(classification_report(actuals, predictions, target_names=[id2char[s] for s in set(actuals + predictions)]))

# Calculate and print the accuracy
accuracy = accuracy_score(actuals, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")

              precision    recall  f1-score   support

           A       0.00      0.00      0.00         1
           C       0.57      1.00      0.73         4
           D       0.00      0.00      0.00         0
           E       0.20      0.14      0.17        14
           F       0.00      0.00      0.00         1
           G       0.52      0.42      0.47        26
           H       0.33      1.00      0.50         1
           I       0.25      0.17      0.20         6
           J       0.22      0.25      0.24         8
           K       0.42      0.73      0.53        11
           L       0.00      0.00      0.00         1
           M       0.83      0.56      0.67        18
           N       0.00      0.00      0.00         2
           P       0.86      0.67      0.75        27
           Q       1.00      1.00      1.00         1
           R       0.44      0.55      0.49        31
           S       0.39      0.67      0.49        33
           T       0.67    

/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

In [22]:
predictions, actuals = predict(model, test_loader)

# Print the classification report
print(classification_report(actuals, predictions, target_names=[id2char[s] for s in set(actuals + predictions)]))

# Calculate and print the accuracy
accuracy = accuracy_score(actuals, predictions)
print(f"Test Accuracy: {accuracy:.4f}")


              precision    recall  f1-score   support

           A       1.00      0.50      0.67         2
           C       0.00      0.00      0.00         0
           E       0.14      0.67      0.24         3
           F       0.00      0.00      0.00         0
           G       0.80      0.89      0.84        27
           H       0.00      0.00      0.00         0
           I       0.50      1.00      0.67         1
           J       0.88      0.78      0.82         9
           K       0.92      1.00      0.96        12
           M       0.81      0.89      0.85        28
           P       0.95      0.76      0.84       111
           Q       0.00      0.00      0.00         0
           R       0.89      0.73      0.80        92
           S       0.40      0.71      0.52        24
           T       1.00      0.79      0.88        14
           W       0.00      0.00      0.00         0
           X       0.00      0.00      0.00         0
           1       1.00    

/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samp

In [23]:

def create_and_train_model(input_size, hidden_size, output_size, num_hidden_layers, embedding_dim, train_loader, val_loader, learning_rate, num_epochs, model_save_path):
    # Create the model instance
    model = SimpleLSTM(input_size, hidden_size, output_size, num_hidden_layers, embedding_dim)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Initialize the best validation loss to infinity at the start
    best_val_loss = float('inf')
    patience = 5  # Number of epochs to wait for improvement before stopping
    epochs_no_improve = 0  # Track the number of epochs with no improvement

    # Training function
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for batch_idx, (data, targets) in enumerate(train_loader):
            # Forward pass
            scores = model(data)
            loss = criterion(scores, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data, targets in val_loader:
                scores = model(data)
                loss = criterion(scores, targets)
                val_loss += loss.item()

        val_loss /= len(val_loader)

        # Print training and validation loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_save_path)
            print(f"Saved model with validation loss: {best_val_loss:.4f}")
            epochs_no_improve = 0  # Reset counter
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs.")
                break  # Early stopping

    # Return the trained model and its best validation loss
    return model, best_val_loss


In [25]:
best_val_loss = float('inf')
best_model_hyperparams = {}
best_model = None

for num_hidden_layers in num_hidden_layers_list:
    for embedding_dim in embedding_dims:
        for hidden_size in hidden_sizes:
            for lr in learning_rates:
                print(f"Training with hidden_size: {hidden_size}, learning_rate: {lr}, embedding_dim: {embedding_dim}, num_hidden_layers: {num_hidden_layers}")
                
                model_save_path = f"../trained_models/model_hs{hidden_size}_lr{lr}_ed{embedding_dim}_nhl{num_hidden_layers}.pth"
                model, val_loss = create_and_train_model(input_size, hidden_size, output_size, num_hidden_layers, embedding_dim, train_loader, val_loader, lr, num_epochs, model_save_path)
                
                print(f"Validation Loss: {val_loss}")
                
                if val_loss < best_val_loss: ### <<<<<<<<<<< Loss based search
                    best_val_loss = val_loss
                    best_model = model
                    best_model_hyperparams = {'hidden_size': hidden_size, 'learning_rate': lr, 'embedding_dim': embedding_dim, 'num_hidden_layers': num_hidden_layers}
                    # Optionally, save the best model
                    torch.save(best_model.state_dict(), model_save_path)
                    predictions, actuals = predict(model, val_loader)
                    
                    # Calculate and print the accuracy
                    accuracy = accuracy_score(actuals, predictions)
                    print(f"===== Accuracy {accuracy:.4f}.")

print(f"Best Model Hyperparameters: {best_model_hyperparams}, Lowest Validation Loss: {best_val_loss}")


Training with hidden_size: 128, learning_rate: 0.0003, embedding_dim: 20, num_hidden_layers: 1
Epoch [1/10], Training Loss: 3.4045, Validation Loss: 3.2611
Saved model with validation loss: 3.2611
Epoch [2/10], Training Loss: 2.9217, Validation Loss: 2.6962
Saved model with validation loss: 2.6962
Epoch [3/10], Training Loss: 2.6113, Validation Loss: 2.5633
Saved model with validation loss: 2.5633
Epoch [4/10], Training Loss: 2.5167, Validation Loss: 2.4794
Saved model with validation loss: 2.4794
Epoch [5/10], Training Loss: 2.4488, Validation Loss: 2.4074
Saved model with validation loss: 2.4074
Epoch [6/10], Training Loss: 2.3914, Validation Loss: 2.3540
Saved model with validation loss: 2.3540
Epoch [7/10], Training Loss: 2.3445, Validation Loss: 2.3169
Saved model with validation loss: 2.3169
Epoch [8/10], Training Loss: 2.3008, Validation Loss: 2.2665
Saved model with validation loss: 2.2665
Epoch [9/10], Training Loss: 2.2612, Validation Loss: 2.2429
Saved model with validation 

In [26]:
# Assuming 'model' is your trained SimpleRNN model
predictions, actuals = predict(best_model, val_loader)

# Print the classification report
print(classification_report(actuals, predictions, target_names=[id2char[s] for s in set(actuals + predictions)]))

# Calculate and print the accuracy
accuracy = accuracy_score(actuals, predictions)
print(f"Accuracy: {accuracy:.4f}")


              precision    recall  f1-score   support

           A       0.00      0.00      0.00         1
           C       0.33      0.25      0.29         4
           E       0.14      0.07      0.10        14
           F       0.00      0.00      0.00         1
           G       0.77      0.38      0.51        26
           H       0.50      1.00      0.67         1
           I       0.33      0.17      0.22         6
           J       0.25      0.12      0.17         8
           K       0.64      0.64      0.64        11
           L       1.00      1.00      1.00         1
           M       0.87      0.72      0.79        18
           N       0.00      0.00      0.00         2
           P       0.68      0.63      0.65        27
           Q       0.00      0.00      0.00         1
           R       0.38      0.68      0.48        31
           S       0.32      0.70      0.44        33
           T       0.80      1.00      0.89         4
           U       0.00    

/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in label

In [27]:
# Assuming 'model' is your trained SimpleRNN model
predictions, actuals = predict(best_model, test_loader)

# Print the classification report
print(classification_report(actuals, predictions, target_names=[id2char[s] for s in set(actuals + predictions)]))

# Calculate and print the accuracy
accuracy = accuracy_score(actuals, predictions)
print(f"Accuracy: {accuracy:.4f}")


              precision    recall  f1-score   support

           A       1.00      0.50      0.67         2
           C       0.00      0.00      0.00         0
           E       0.38      1.00      0.55         3
           G       0.96      0.96      0.96        27
           H       0.00      0.00      0.00         0
           I       1.00      1.00      1.00         1
           J       0.71      0.56      0.63         9
           K       1.00      1.00      1.00        12
           M       0.87      0.96      0.92        28
           P       0.98      0.81      0.89       111
           R       0.85      0.93      0.89        92
           S       0.52      0.96      0.68        24
           T       1.00      1.00      1.00        14
           1       1.00      0.76      0.87        17
           2       0.88      0.56      0.69        39

    accuracy                           0.85       379
   macro avg       0.74      0.73      0.71       379
weighted avg       0.89   

/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samp

In [28]:

def objective(trial):
    # Define the hyperparameter search space
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 3)
    embedding_dim = trial.suggest_categorical('embedding_dim', [50, 100, 128])
    hidden_size = trial.suggest_categorical('hidden_size', [128, 256])
    lr = trial.suggest_categorical('lr', [0.0004, 0.0003, 0.0002])
    num_epochs = trial.suggest_int('num_epochs', 40, 70)

    # Train the model using the suggested hyperparameters
    model_save_path = f"../trained_models/model_trial_{trial.number}.pth"
    model, val_loss = create_and_train_model(input_size, hidden_size, output_size, num_hidden_layers, embedding_dim, train_loader, val_loader, lr, num_epochs, model_save_path)

    predictions, actuals = predict(model, val_loader)
    accuracy = accuracy_score(actuals, predictions)
    print(f"===== Accuracy {accuracy:.4f}.")

    return val_loss


In [29]:
n_trials=50

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)  # Set n_trials to the number of trials you want to run

best_trial = study.best_trial
print(f"Best trial: {best_trial.number}")
print(f"Best hyperparameters: {best_trial.params}")
print(f"Best validation loss: {best_trial.value}")

# Load the best model
best_model_path = f"../trained_models/model_trial_{best_trial.number}.pth"
best_model = SimpleLSTM(input_size, best_trial.params['hidden_size'], output_size, best_trial.params['num_hidden_layers'], best_trial.params['embedding_dim'])
best_model.load_state_dict(torch.load(best_model_path))


[I 2023-11-22 08:39:42,491] A new study created in memory with name: no-name-7739387b-cc86-4cd6-ba6d-765e293d9afc


Epoch [1/64], Training Loss: 3.2848, Validation Loss: 2.8551
Saved model with validation loss: 2.8551
Epoch [2/64], Training Loss: 2.7112, Validation Loss: 2.6269
Saved model with validation loss: 2.6269
Epoch [3/64], Training Loss: 2.5568, Validation Loss: 2.4600
Saved model with validation loss: 2.4600
Epoch [4/64], Training Loss: 2.4400, Validation Loss: 2.3569
Saved model with validation loss: 2.3569
Epoch [5/64], Training Loss: 2.3572, Validation Loss: 2.2681
Saved model with validation loss: 2.2681
Epoch [6/64], Training Loss: 2.2794, Validation Loss: 2.2412
Saved model with validation loss: 2.2412
Epoch [7/64], Training Loss: 2.2215, Validation Loss: 2.1731
Saved model with validation loss: 2.1731
Epoch [8/64], Training Loss: 2.1598, Validation Loss: 2.1259
Saved model with validation loss: 2.1259
Epoch [9/64], Training Loss: 2.0980, Validation Loss: 2.0814
Saved model with validation loss: 2.0814
Epoch [10/64], Training Loss: 2.0491, Validation Loss: 2.0352
Saved model with val

[I 2023-11-22 08:39:52,059] Trial 0 finished with value: 1.7795489132404327 and parameters: {'num_hidden_layers': 2, 'embedding_dim': 100, 'hidden_size': 128, 'lr': 0.0003, 'num_epochs': 64}. Best is trial 0 with value: 1.7795489132404327.


Epoch [37/64], Training Loss: 1.2770, Validation Loss: 1.8071
Early stopping triggered after 37 epochs.
===== Accuracy 0.4510.
Epoch [1/67], Training Loss: 2.9529, Validation Loss: 2.4833
Saved model with validation loss: 2.4833
Epoch [2/67], Training Loss: 2.3542, Validation Loss: 2.1723
Saved model with validation loss: 2.1723
Epoch [3/67], Training Loss: 2.1672, Validation Loss: 2.0479
Saved model with validation loss: 2.0479
Epoch [4/67], Training Loss: 2.0359, Validation Loss: 1.9955
Saved model with validation loss: 1.9955
Epoch [5/67], Training Loss: 1.9461, Validation Loss: 1.9003
Saved model with validation loss: 1.9003
Epoch [6/67], Training Loss: 1.8738, Validation Loss: 1.8973
Saved model with validation loss: 1.8973
Epoch [7/67], Training Loss: 1.8028, Validation Loss: 1.8518
Saved model with validation loss: 1.8518
Epoch [8/67], Training Loss: 1.7476, Validation Loss: 1.8083
Saved model with validation loss: 1.8083
Epoch [9/67], Training Loss: 1.6937, Validation Loss: 1.7

[I 2023-11-22 08:39:59,497] Trial 1 finished with value: 1.7001923322677612 and parameters: {'num_hidden_layers': 1, 'embedding_dim': 100, 'hidden_size': 256, 'lr': 0.0004, 'num_epochs': 67}. Best is trial 1 with value: 1.7001923322677612.


...

[I 2023-11-22 08:49:10,449] Trial 48 finished with value: 1.68478125333786 and parameters: {'num_hidden_layers': 1, 'embedding_dim': 100, 'hidden_size': 256, 'lr': 0.0004, 'num_epochs': 59}. Best is trial 33 with value: 1.6513267159461975.


Epoch [24/59], Training Loss: 1.1074, Validation Loss: 1.7284
Early stopping triggered after 24 epochs.
===== Accuracy 0.4863.
Epoch [1/53], Training Loss: 2.9255, Validation Loss: 2.3923
Saved model with validation loss: 2.3923
Epoch [2/53], Training Loss: 2.3214, Validation Loss: 2.1058
Saved model with validation loss: 2.1058
Epoch [3/53], Training Loss: 2.1220, Validation Loss: 2.0029
Saved model with validation loss: 2.0029
Epoch [4/53], Training Loss: 2.0066, Validation Loss: 1.9281
Saved model with validation loss: 1.9281
Epoch [5/53], Training Loss: 1.9153, Validation Loss: 1.8912
Saved model with validation loss: 1.8912
Epoch [6/53], Training Loss: 1.8366, Validation Loss: 1.8407
Saved model with validation loss: 1.8407
Epoch [7/53], Training Loss: 1.7683, Validation Loss: 1.7902
Saved model with validation loss: 1.7902
Epoch [8/53], Training Loss: 1.7037, Validation Loss: 1.7696
Saved model with validation loss: 1.7696
Epoch [9/53], Training Loss: 1.6529, Validation Loss: 1.7

[I 2023-11-22 08:49:16,463] Trial 49 finished with value: 1.7109911739826202 and parameters: {'num_hidden_layers': 1, 'embedding_dim': 128, 'hidden_size': 256, 'lr': 0.0004, 'num_epochs': 53}. Best is trial 33 with value: 1.6513267159461975.


Epoch [17/53], Training Loss: 1.2999, Validation Loss: 1.7165
Early stopping triggered after 17 epochs.
===== Accuracy 0.4863.
Best trial: 33
Best hyperparameters: {'num_hidden_layers': 1, 'embedding_dim': 100, 'hidden_size': 256, 'lr': 0.0004, 'num_epochs': 55}
Best validation loss: 1.6513267159461975


<All keys matched successfully>

In [30]:
# Assuming 'model' is your trained SimpleRNN model
predictions, actuals = predict(best_model, val_loader)

# Print the classification report
print(classification_report(actuals, predictions, target_names=[id2char[s] for s in set(actuals + predictions)]))

# Calculate and print the accuracy
accuracy = accuracy_score(actuals, predictions)
print(f"Accuracy: {accuracy:.4f}")


              precision    recall  f1-score   support

           A       0.00      0.00      0.00         1
           C       0.60      0.75      0.67         4
           E       0.33      0.14      0.20        14
           F       0.00      0.00      0.00         1
           G       0.71      0.38      0.50        26
           H       0.33      1.00      0.50         1
           I       0.20      0.17      0.18         6
           J       0.27      0.38      0.32         8
           K       0.47      0.64      0.54        11
           L       0.00      0.00      0.00         1
           M       0.86      0.67      0.75        18
           N       0.00      0.00      0.00         2
           P       0.68      0.63      0.65        27
           Q       0.00      0.00      0.00         1
           R       0.41      0.77      0.54        31
           S       0.46      0.67      0.54        33
           T       0.80      1.00      0.89         4
           U       0.00    

/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in label

In [31]:
# Assuming 'model' is your trained SimpleRNN model
predictions, actuals = predict(best_model, test_loader)

# Print the classification report
print(classification_report(actuals, predictions, target_names=[id2char[s] for s in set(actuals + predictions)]))

# Calculate and print the accuracy
accuracy = accuracy_score(actuals, predictions)
print(f"Accuracy: {accuracy:.4f}")


              precision    recall  f1-score   support

           A       1.00      1.00      1.00         2
           E       0.50      1.00      0.67         3
           F       0.00      0.00      0.00         0
           G       1.00      0.93      0.96        27
           H       0.00      0.00      0.00         0
           I       0.50      1.00      0.67         1
           J       0.88      0.78      0.82         9
           K       1.00      0.92      0.96        12
           M       0.90      0.93      0.91        28
           P       0.96      0.80      0.87       111
           Q       0.00      0.00      0.00         0
           R       0.86      0.92      0.89        92
           S       0.66      0.79      0.72        24
           T       1.00      1.00      1.00        14
           Z       0.00      0.00      0.00         0
           1       1.00      0.65      0.79        17
           2       0.72      0.67      0.69        39
           5       0.00    

/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samp